In [ ]:
!pip install TwitterAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for TwitterAPI: filename=TwitterAPI-2.8.1-py3-none-any.whl size=14886 sha256=dbcb7fef4f4b7441f10b3282869fe5797cf033f3efe365e1ea9d1e75e62fd36c
  Stored in directory: /root/.cache/pip/wheels/e8/f4/9d/1aa8a6ce119f9cff47ed030e146498ea6f8707907f09f3ffc6
Successfully built TwitterAPI


In [ ]:
# INSTALL TWITTER API (!pip install TwitterAPI) and import necessary libraries/packages
# TwitterAPI Documentation here: https://github.com/geduldig/TwitterAPI/
from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError, TwitterConnectionError, TwitterPager
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#os.getcwd()

Mounted at /content/drive


In [ ]:
import sys 
import os

path_to_utils='/content/drive/My Drive/NLP'
sys.path.append(path_to_utils)

In [ ]:
os.chdir(path_to_utils)

print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1qJ8h8Q6C6t_FWn4R8pQeH813twKX9HgP/NLP


In [ ]:
consumer_key= "no1wMGrU6ZwbynZCvn1DYEfEx"
consumer_secret= "UEEHTvHiA8LOvRr68MXHBlQuAiBgWYwV6R3Z6EPuxHOVLvPvFT"
access_token_key= "1597624861484060673-i1aZpcKbEnKYGgCidQ8Zg86xggXJkc"
access_token_secret= "3EBEmSVCE2VEgVnoo0Yuz3Nh9BwXMStyijxBzW2pzT0tc"

api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret, api_version='2')

In [ ]:
reply1_tweets = pd.read_csv('train_reply1_tweets.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
reply1_tweets.columns

Index(['id', 'conversation_id', 'referenced_tweets.replied_to.id',
       'referenced_tweets.retweeted.id', 'referenced_tweets.quoted.id',
       'edit_history_tweet_ids', 'edit_controls.edits_remaining',
       'edit_controls.editable_until', 'edit_controls.is_edit_eligible',
       'author_id', 'in_reply_to_user_id', 'retweeted_user_id',
       'quoted_user_id', 'created_at', 'text', 'lang', 'source',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'public_metrics.reply_count', 'public_metrics.retweet_count',
       'reply_settings', 'possibly_sensitive', 'withheld.scope',
       'withheld.copyright', 'withheld.country_codes', 'entities.annotations',
       'entities.cashtags', 'entities.hashtags', 'entities.mentions',
       'entities.urls', 'context_annotations', 'attachments.media',
       'attachments.media_keys', 'attachments.poll.duration_minutes',
       'attachments.poll.end_datetime', 'attachments.poll.id',
       'attachments.poll.options', 'attachm

In [ ]:
reply1_tweets.conversation_id

0        1223752255846912000
1        1223748267609030659
2        1223739174160928773
3        1223737953291128837
4        1223736784040923136
                ...         
15417    1221509351678058502
15418    1221506691461722112
15419    1221501747606237190
15420    1221493715467931648
15421    1221488390681571329
Name: conversation_id, Length: 15422, dtype: int64

In [55]:
reply1_tweets['author.username']

0         DesignationSix
1        PriscillaAnyabu
2              SecNorman
3             larryelder
4          MatttSkidmore
              ...       
15417        NextIrishGE
15418              K24Tv
15419        NextIrishGE
15420        localnotail
15421          BobPersis
Name: author.username, Length: 15422, dtype: object

In [ ]:
conv_ids = ['1223752255846912000'] 

In [ ]:
class TreeNode:
	def __init__(self, data):
		"""data is a tweet's json object"""
		self.data = data
		self.children = []

	def id(self):
		"""a node is identified by its author"""
		return self.data['author_id']

	def reply_to(self):
		"""the reply-to user is the parent of the node"""
		return self.data['in_reply_to_user_id']

	def find_parent_of(self, node):
		"""append a node to the children of it's reply-to user"""
		if node.reply_to() == self.id():
			self.children.append(node)
			return True
		for child in self.children:
			if child.find_parent_of(node):
				return True
		return False

	def print_tree(self, level):
		"""level 0 is the root node, then incremented for subsequent generations"""
		# print(f'{level*"_"}{level}: {self.id()}')
		level += 1
		for child in self.children:
			child.print_tree(level)

	def list_l1(self):
		conv_id = []
		child_id = []
		text = []
		# print(self.data['id'])
		for child in self.children:
			conv_id.append(self.data['id'])
			child_id.append(child.data['id'])
			text.append(child.data['text'])
		return conv_id, child_id, text

In [ ]:
def retrieve_replies(conversation_id):
    try:
        # GET ROOT OF THE CONVERSATION
        r = api.request(f'tweets/:{conversation_id}',
                        {
                            'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id'
                        })
        #print(r)
        for item in r:
            root = TreeNode(item)
            # print(f'ROOT {root.id()}')

        # GET ALL REPLIES IN CONVERSATION

        pager = TwitterPager(api, 'tweets/search/recent',
                             {
                                 'query': f'conversation_id:{conversation_id}',
                                 'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id'
                             })

        orphans = []

        for item in pager.get_iterator(wait=2):
            node = TreeNode(item)
            # print(f'{node.id()} => {node.reply_to()}')
            # COLLECT ANY ORPHANS THAT ARE NODE'S CHILD
            orphans = [orphan for orphan in orphans if not node.find_parent_of(orphan)]
            # IF NODE CANNOT BE PLACED IN TREE, ORPHAN IT UNTIL ITS PARENT IS FOUND
            if not root.find_parent_of(node):
                orphans.append(node)

        conv_id, child_id, text = root.list_l1()
#         print('\nTREE...')
# 	    root.print_tree(0)

        assert len(orphans) == 0, f'{len(orphans)} orphaned tweets'

    except TwitterRequestError as e:
        print(e.status_code)
        for msg in iter(e):
            print(msg)

    except TwitterConnectionError as e:
        print(e)

    except Exception as e:
        print(e)
    
    return conv_id, child_id, text



"""
Retrieves replies for a list of conversation ids (conv_ids
Returns a dataframe with columns [conv_id, child_id, text] tuple which shows every reply's tweet_id and text in the last two columns
"""
def reply_thread_maker(conv_ids):
    conv_id = []
    child_id = []
    text = []
    for id in conv_ids:
        conv_id1, child_id1, text1 = retrieve_replies(id)
        conv_id.extend(conv_id1)
        child_id.extend(child_id1)
        text.extend(text1)

    replies_data = {'conversation_id' : conv_id,
               'child_tweet_id': child_id,
               'tweet_text' : text}

    replies= pd.DataFrame(replies_data)
    return replies

In [ ]:
replies = reply_thread_maker(conv_ids)

In [ ]:
replies

,conversation_id,child_tweet_id,tweet_text


In [56]:
r = api.request(f'tweets/:{1223752255846912000}',
                        {
                            'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id'
                        })
for item in r:
            root = TreeNode(item)
            #print(f'ROOT {root.id()}')
            pager = TwitterPager(api, 'tweets/search/recent',
                             {
                                 'query': f'conversation_id:{1223752255846912000}',
                                 'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id'
                             })
            
            for item in pager.get_iterator(wait=2):
              node = TreeNode(item)
              print(f'{node.id()} => {node.reply_to()}')
              # COLLECT ANY ORPHANS THAT ARE NODE'S CHILD
              orphans = [orphan for orphan in orphans if not node.find_parent_of(orphan)]
              # IF NODE CANNOT BE PLACED IN TREE, ORPHAN IT UNTIL ITS PARENT IS FOUND
              if not root.find_parent_of(node):
                  orphans.append(node)
            

In [53]:
pager = TwitterPager(api, 'tweets/search/recent',
                             {
                                 'query': f'conversation_id:{1223752255846912000}',
                                 'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id'
                             })

orphans = []

for item in pager.get_iterator(wait=2):
    node = TreeNode(item)
    print(f'{node.id()} => {node.reply_to()}')
    # COLLECT ANY ORPHANS THAT ARE NODE'S CHILD
    orphans = [orphan for orphan in orphans if not node.find_parent_of(orphan)]
    # IF NODE CANNOT BE PLACED IN TREE, ORPHAN IT UNTIL ITS PARENT IS FOUND
    if not root.find_parent_of(node):
        orphans.append(node)

In [54]:
orphans

[]

In [ ]:
for item in r:
  root = TreeNode(item)

{'id': '1223752255846912000', 'conversation_id': '1223752255846912000', 'author_id': '969138694308421632', 'edit_history_tweet_ids': ['1223752255846912000'], 'text': 'The Fox Corporation (The owners of Fox News) are trying to bully Roku by using the Superbowl as a bargaining chip. Are you going to watch the Fox sponsored Superbowl?\n\nHere are some Fox News sponsors\n\nNestle\nPurina\nDrumstick\nArbys\nMcDonalds\nDisney\nUniversalPictures\nRedLobster\nGSK', 'created_at': '2020-02-01T23:37:28.000Z'}


In [ ]:
pager = TwitterPager(api, 'tweets/search/recent',
                             {
                                 'query': f'conversation_id:{1223752255846912000}',
                                 'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id'
                             })
pager

In [ ]:
for item in pager.get_iterator(wait=2):
  node = TreeNode(item)
  print(node)